## Imports

In [1]:
import os
import json

parent_path = os.path.abspath("./../")
if os.path.basename(parent_path) == 'vdp':
    os.chdir(parent_path)

from glob import glob
import vdp
import re
import copy
import numpy as np

from tqdm.auto import tqdm


from vdp.vocabulary import FOSort, FOFunction
from vdp.fomodel import FOElement, FOModel
from vdp.exceptions import PartialInterpretationException




## Generate config.json files

In [2]:
# define the scene graph and the fo model generation configs.
sgconfig = {
"output_dir": "./../data/sg_processed",
"glove_path": "./../../checkpoints/glove",
"model_path": "./../../checkpoints/model",
"sg_tools_rel_path": "./tools/relation_test_net.py",
"sg_config_path": "./configs/e2e_relation_X_101_32_8_FPN_1x.yaml",
"cuda_device_port": 0,
'n_proc': 1
}

foconfig = {
    "input_dir": "./data/sg_processed",
    "output_dir": "./data/sg_ir",
    "rel_topk": 20, 
    "box_topk": 20
}

In [3]:
# write config files to memory.
vdp_puzzles =  glob("./data/full_raw/*")
os.makedirs("./config", exist_ok=True)
for folder in vdp_puzzles:
    train = glob(folder + "/train*")
    test  = glob(folder + "/test*")
    name = os.path.basename(folder)
    config = {
        'name' : name,
        'train' : list(train),
        'test' : list(test),
        'sg_config' : sgconfig,
        'fo_config' : foconfig
    }
    json.dump(config, open(f"./config/{name}.json", 'w'))
    

    

## YOLO $\to$ IR

In [24]:
def yolo_to_ir(yolo_folders, output_folder):
    for folder_path in yolo_folders:
        name = os.path.basename(folder_path)
        output_dir =f"{output_folder}/{name}" 
        os.makedirs(output_dir, exist_ok=True)
        fo_models = vdp.utils.get_yolo_fo_models(folder_path)
        for batch in ['test', 'train']:
            batch_path = f"{output_dir}/{batch}"
            os.makedirs(batch_path, exist_ok=True)
            for img_path, fo_model in fo_models:
                if os.path.basename(img_path).split("_")[0] == batch:
                    json_name = os.path.basename(img_path).replace('.jpg', '_model.json')
                    output_json = f"{batch_path}/{json_name}"
                    with open(output_json, 'w') as fp:
                        json.dump(fo_model, fp, indent=2)


yolo_to_ir(glob("./vdp_smt/*/*"), './data/yolo_ir')


## SGG $\to$ IR

In [25]:
def sgg_to_ir(input_dirs, output_dir, box_topk, rel_topk, raw_img_dir):
    
    for input_dir in input_dirs:
        name = os.path.basename(input_dir)
        for batch_dir in ['train', 'test']:
            sg_input_dir = input_dir + f"/{batch_dir}"
            fo_output_dir = output_dir + f"/{name}/{batch_dir}"
            os.makedirs(fo_output_dir, exist_ok=True)
            fo_models = vdp.utils.get_sgg_fo_models(sg_input_dir, box_topk=box_topk, rel_topk=rel_topk, raw_img_dir=raw_img_dir)
            print(f"{name}, {batch_dir}, {len(fo_models)}")
            for img_path, fo_model in fo_models:
                output_json = f"{fo_output_dir}/{os.path.basename(img_path).replace('.jpg', '_model.json')}"
                with open(output_json, 'w') as fp:
                    json.dump(fo_model, fp, indent=2)

        

sgg_to_ir(glob("./data/sg_processed/*"), "./data/sg_ir", 20, 20, "./data/raw")

alldogsonsofas_abc-23d, train, 3
alldogsonsofas_abc-23d, test, 3
kitchen_abc-1.12.1e, train, 3
kitchen_abc-1.12.1e, test, 3
catontv_ab-1.22.1c, train, 2
catontv_ab-1.22.1c, test, 3
topmost_abc-13e, train, 3
topmost_abc-13e, test, 3
2on1_abe-c123, train, 3
2on1_abe-c123, test, 4
sleeping_abc-1.22.2d, train, 3
sleeping_abc-1.22.2d, test, 3
desktop_abc-1.12.13.1e, train, 3
desktop_abc-1.12.13.1e, test, 4
populatedbus_abc-1.12.2d, train, 3
populatedbus_abc-1.12.2d, test, 3
dogherded_abc-1.12.23.1d, train, 3
dogherded_abc-1.12.23.1d, test, 4
umbrella_abc-135d, train, 3
umbrella_abc-135d, test, 4
pplandties_abc-1.22.1e, train, 3
pplandties_abc-1.22.1e, test, 3
parkmeter_abc-234d, train, 3
parkmeter_abc-234d, test, 4
tvon_abc-13d, train, 3
tvon_abc-13d, test, 3
diagonal_ab-14c, train, 2
diagonal_ab-14c, test, 3
pplwearingties_abc-1.12.13.1e, train, 3
pplwearingties_abc-1.12.13.1e, test, 4
samelevel_ab-17c, train, 2
samelevel_ab-17c, test, 3
allcatsonsofas_abc-1.12f, train, 3
allcatsonsofas_ab

## IR $\to$ smt

In [2]:
with open("./data/yolo_ir/2on1_abc-d123/train/train_a_model.json", "r") as f:
    ir = json.load(f)
ir

{'sorts': ['object', 'label'],
 'predicates': {'right': ['object', 'object'],
  'above': ['object', 'object'],
  'left': ['object', 'object'],
  'below': ['object', 'object'],
  'within': ['object', 'object'],
  'has_label': ['object', 'label']},
 'elements': {'object': ['0_sofa', '1_cat'], 'label': ['cat', 'sofa']},
 'interpretation': {'right': [['0_sofa', '1_cat']],
  'above': [['0_sofa', '1_cat']],
  'left': [['1_cat', '0_sofa']],
  'below': [['1_cat', '0_sofa']],
  'within': [['1_cat', '0_sofa']],
  'has_label': [['0_sofa', 'sofa'], ['1_cat', 'cat']]},
 'raw': {'rel_labels': [['0', 'sofa', '1', 'cat', 'right', '1.0'],
   ['0', 'sofa', '1', 'cat', 'above', '1.0'],
   ['1', 'cat', '0', 'sofa', 'left', '1.0'],
   ['1', 'cat', '0', 'sofa', 'below', '1.0'],
   ['1', 'cat', '0', 'sofa', 'within', '1.0']],
  'var_const_map': {'0_sofa': 'sofa', '1_cat': 'cat'},
  'scores': [['0_sofa', '1_cat', '1.0'],
   ['0_sofa', '1_cat', '1.0'],
   ['1_cat', '0_sofa', '1.0'],
   ['1_cat', '0_sofa', '1.0

In [3]:
vdp_sorts = dict()
vdp_constants = dict()

assert('label' in ir['sorts'])
assert('object' in ir['sorts'])
for sort in ir['sorts']:
    vdp_sorts[sort] = FOSort(sort)
    
    
    
for label in ir['elements']['label']:
    if label not in vdp_constants:
        vdp_constants[label] = FOFunction(label, tuple([vdp_sorts['label']]))


unique_labels = list(ir['elements']['label']) # @TODO REPLACE!!!!
unique_labels

['cat', 'sofa']

In [4]:
relation_labels = list(ir['predicates'].keys())

vdp_relations = dict()

for rl in relation_labels:
    sig = tuple([*ir['predicates'][rl], 'bool'])
    vdp_relations[rl] = FOFunction(rl, sig)


vdp_relations


{'right': <vdp.vocabulary.FOFunction at 0x11d67d210>,
 'above': <vdp.vocabulary.FOFunction at 0x11d67d190>,
 'left': <vdp.vocabulary.FOFunction at 0x11d67d1d0>,
 'below': <vdp.vocabulary.FOFunction at 0x11d67d250>,
 'within': <vdp.vocabulary.FOFunction at 0x11d67d290>,
 'has_label': <vdp.vocabulary.FOFunction at 0x11d67d2d0>}

In [5]:
vdp_vocabulary = (set([vdp_sorts['object'], vdp_sorts['label']]), set(list(vdp_constants.values()) + list(vdp_relations.values())))

In [6]:
vdp_vocabulary = (set([vdp_sorts['object'], vdp_sorts['label']]), set(list(vdp_constants.values()) + list(vdp_relations.values())))
vdp_objects = {obj : FOElement(obj, vdp_sorts['object']) for obj, lab in ir['interpretation']['has_label']}
vdp_labels = {lab : FOElement(lab, vdp_sorts['label']) for obj, lab in ir['interpretation']['has_label']}

vdp_elements = {'Object': set(vdp_objects.values()), 'Label': set(vdp_labels.values())}

In [7]:


labelconst_interpretation = {const : vdp_labels[label] for label, const in zip(unique_labels, list(vdp_constants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x11d67f590>: <vdp.fomodel.FOElement at 0x11d68db90>,
 <vdp.vocabulary.FOFunction at 0x11d67f650>: <vdp.fomodel.FOElement at 0x11d68dbd0>}

In [8]:
vdp_relations

{'right': <vdp.vocabulary.FOFunction at 0x11d67d210>,
 'above': <vdp.vocabulary.FOFunction at 0x11d67d190>,
 'left': <vdp.vocabulary.FOFunction at 0x11d67d1d0>,
 'below': <vdp.vocabulary.FOFunction at 0x11d67d250>,
 'within': <vdp.vocabulary.FOFunction at 0x11d67d290>,
 'has_label': <vdp.vocabulary.FOFunction at 0x11d67d2d0>}

In [9]:
ir_interpretation = ir['interpretation']
rel_interpretation = dict()
for rel_sym, rel in vdp_relations.items():
    if 'has_label' in rel_sym:
        rel_interpretation[rel] = {(obj, lab) : [obj_sym, lab_sym] in ir_interpretation[rel_sym] for obj_sym, obj in vdp_objects.items() for lab_sym, lab in vdp_labels.items()}
    else:
        rel_interpretation[rel] = {(obj1, obj2) : [obj1_sym, obj2_sym] in ir_interpretation[rel_sym]  for obj1_sym, obj1 in vdp_objects.items() for obj2_sym, obj2 in vdp_objects.items() if obj1_sym != obj2_sym}

        
vdp_interpretation = {**labelconst_interpretation, **rel_interpretation}
model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)


In [10]:
vdp_interpretation = {**labelconst_interpretation, **rel_interpretation}


In [11]:
model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)


In [12]:
vdp_objects.keys()

dict_keys(['0_sofa', '1_cat'])

In [2]:
# combining everything before this...
def ir_to_model(ir_path):
    with open(ir_path, "r") as f:
        ir = json.load(f)
    vdp_sorts = dict()
    vdp_constants = dict()
    vdp_relations = dict()
    rel_interpretation = dict()

    assert('label' in ir['sorts'])
    assert('object' in ir['sorts'])
    for sort in ir['sorts']:
        vdp_sorts[sort] = FOSort(sort)
    for label in ir['elements']['label']:
        if label not in vdp_constants:
            vdp_constants[label] = FOFunction(label, tuple([vdp_sorts['label']]))

    unique_labels = list(ir['elements']['label']) # @TODO REPLACE!!!!    
    relation_labels = list(ir['predicates'].keys())

    for rl in relation_labels:
        sig = tuple([*ir['predicates'][rl], 'bool'])
        vdp_relations[rl] = FOFunction(rl, sig)

    vdp_vocabulary = (set([vdp_sorts['object'], vdp_sorts['label']]), set(list(vdp_constants.values()) + list(vdp_relations.values())))
    vdp_objects = {obj : FOElement(obj, vdp_sorts['object']) for obj, lab in ir['interpretation']['has_label']}
    vdp_labels = {lab : FOElement(lab, vdp_sorts['label']) for obj, lab in ir['interpretation']['has_label']}
    vdp_elements = {'Object': set(vdp_objects.values()), 'Label': set(vdp_labels.values())}
    labelconst_interpretation = {const : vdp_labels[label] for label, const in zip(unique_labels, list(vdp_constants.values()))}

    ir_interpretation = ir['interpretation']
    for rel_sym, rel in vdp_relations.items():
        if 'has_label' in rel_sym:
            rel_interpretation[rel] = {(obj, lab) : [obj_sym, lab_sym] in ir_interpretation[rel_sym] for obj_sym, obj in vdp_objects.items() for lab_sym, lab in vdp_labels.items()}
        else:
            rel_interpretation[rel] = {(obj1, obj2) : [obj1_sym, obj2_sym] in ir_interpretation[rel_sym]  for obj1_sym, obj1 in vdp_objects.items() for obj2_sym, obj2 in vdp_objects.items() if obj1_sym != obj2_sym}

    vdp_interpretation = {**labelconst_interpretation, **rel_interpretation}
    vdp_model = FOModel(vdp_vocabulary, vdp_elements, vdp_interpretation)
    return vdp_sorts, vdp_relations, vdp_constants, vdp_objects, vdp_model


In [3]:
# testing
sorts, relations, constants, objects, model = ir_to_model("./data/yolo_ir/2on1_abc-d123/train/train_a_model.json")
tst = FOElement('x', sorts['object'])
cat_is_cat = model.interpret(term=[relations['has_label'], [tst], [constants['cat']]], interpretation_extension={tst: objects['1_cat']})
cat_is_sofa = model.interpret(term=[relations['has_label'], [tst], [constants['cat']]], interpretation_extension={tst: objects['0_sofa']})

cat_is_cat, cat_is_sofa

(True, False)

## Clean up the vdp results

In [83]:
def recurse_rename(dictionary):
    for k, v in dictionary.items():            
        if isinstance(v, dict):
            recurse_rename(v)

        if k == 'has label':
            dictionary['has_label'] = copy.deepcopy(v)
            del dictionary['has label']

        if k == 'has score':
            dictionary['has_score'] = copy.deepcopy(v)
            del dictionary['has score']

    

    
for file in (glob("./data/sg_ir/*/*/*")):
    with open(file) as fp:
        ir = json.load(fp)
    
    recurse_rename(ir)
        
    with open(file, 'w') as fp:
        json.dump(ir, fp)
    
for (dirpath, dirnames, filenames) in os.walk("./data/yolo_ir"):
    for file in filenames:
        pth = os.path.join(dirpath, file)
        with open(pth) as fp:
            ir = json.load(fp)
        recurse_rename(ir)
        with open(pth, 'w') as fp:
            json.dump(ir, fp)    

for (dirpath, dirnames, filenames) in os.walk("./data/yolo_ir"):
    for dirn in dirnames:
        if dirn in ['test', 'train']:
            continue
        pth = os.path.join(dirpath, dirn)
        test_pth = (os.path.join(pth, 'test'))
        train_pth = (os.path.join(pth, 'train'))
        os.makedirs(test_pth, exist_ok=True)
        os.makedirs(train_pth, exist_ok=True)
        for f in glob(os.path.join(pth, "*.json")):
            t = os.path.basename(f).split("_")[0]
            if f"{t}/{t}" not in f:
                p = (os.path.join(os.path.dirname(f), t ,os.path.basename(f)))
                print(f, "->", p)
                os.rename(f, p)


./data/yolo_ir/samelevel_ab-16c/test_1_model.json -> ./data/yolo_ir/samelevel_ab-16c/test/test_1_model.json
./data/yolo_ir/samelevel_ab-16c/test_6_model.json -> ./data/yolo_ir/samelevel_ab-16c/test/test_6_model.json
./data/yolo_ir/samelevel_ab-16c/train_a_model.json -> ./data/yolo_ir/samelevel_ab-16c/train/train_a_model.json
./data/yolo_ir/samelevel_ab-16c/test_c_model.json -> ./data/yolo_ir/samelevel_ab-16c/test/test_c_model.json
./data/yolo_ir/samelevel_ab-16c/train_b_model.json -> ./data/yolo_ir/samelevel_ab-16c/train/train_b_model.json
./data/yolo_ir/tvon_abc-25d/train_c_model.json -> ./data/yolo_ir/tvon_abc-25d/train/train_c_model.json
./data/yolo_ir/tvon_abc-25d/train_a_model.json -> ./data/yolo_ir/tvon_abc-25d/train/train_a_model.json
./data/yolo_ir/tvon_abc-25d/test_5_model.json -> ./data/yolo_ir/tvon_abc-25d/test/test_5_model.json
./data/yolo_ir/tvon_abc-25d/train_b_model.json -> ./data/yolo_ir/tvon_abc-25d/train/train_b_model.json
./data/yolo_ir/tvon_abc-25d/test_2_model.json

# Undocument code

In [7]:

relations = {}
for label1, pred1, bb1 in img_params:
    for label2, pred2, bb2 in img_params:
        if label1 != label2:
            c1 = centroid(bb1)
            c2 = centroid(bb2)
            relations[(label1, label2)] = relate(c1, c2)
        else:
            relations[(label1, label2)] = ['labelOf']
            
labels, preds, bbs = zip(*img_params)

relations


{('bed', 'bed'): ['labelOf'],
 ('bed', 'sofa'): ['right', 'above'],
 ('bed', 'cat'): ['left', 'above'],
 ('sofa', 'bed'): ['left', 'below'],
 ('sofa', 'sofa'): ['labelOf'],
 ('sofa', 'cat'): ['left', 'above'],
 ('cat', 'bed'): ['right', 'below'],
 ('cat', 'sofa'): ['right', 'below'],
 ('cat', 'cat'): ['labelOf']}

In [8]:
vdpobject = FOSort('Object')
vdplabel = FOSort('Label')
vdpbool = FOSort('Bool')


In [9]:
vdpconstants = dict()
for label in labels:
    if label not in vdpconstants:
        vdpconstants[label] = FOFunction(label, tuple([vdplabel]))
    
    
unique_labels = list(vdpconstants.keys()) # @TODO REPLACE!!!!


In [103]:
vdp_objects

{'0_sofa': <vdp.fomodel.FOElement at 0x11e037410>,
 '1_cat': <vdp.fomodel.FOElement at 0x11e037c10>}

True

In [10]:
# on = FOFunction('On', (vdpobject, vdpobject, vdpbool))
relation_labels = ['left', 'right', 'above', 'below']
vdprelations = dict()

for rl in relation_labels:
    vdprelations[rl] = FOFunction(rl, (vdpobject, vdpobject, vdpbool))


vdprelations['labelOf'] = FOFunction('labelOf', (vdpobject, vdplabel, vdpbool))
vdprelations

{'left': <vdp.vocabulary.FOFunction at 0x10889af10>,
 'right': <vdp.vocabulary.FOFunction at 0x10889a250>,
 'above': <vdp.vocabulary.FOFunction at 0x10889ab90>,
 'below': <vdp.vocabulary.FOFunction at 0x10889a790>,
 'labelOf': <vdp.vocabulary.FOFunction at 0x10889ad10>}

In [11]:

vdpvocabulary = (set([vdpobject, vdplabel]), set(list(vdpconstants.values()) + list(vdprelations.values())))

In [12]:
vdpobjects = {label : FOElement(label + "_object", vdpobject) for label in labels}
vdplabels = {label : FOElement(label + "_label", vdplabel) for label in unique_labels}


vdpelements = {'Object': set(vdpobjects.values()), 'Label': set(vdplabels.values())}

In [13]:
labelconst_interpretation = {const : vdplabels[label] for label, const in zip(unique_labels, list(vdpconstants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x10889a650>: <vdp.fomodel.FOElement at 0x111a026d0>,
 <vdp.vocabulary.FOFunction at 0x10889a2d0>: <vdp.fomodel.FOElement at 0x111a02810>,
 <vdp.vocabulary.FOFunction at 0x10889a510>: <vdp.fomodel.FOElement at 0x111a02910>}

In [14]:

    

rel_interpretation = dict()
# for rel_sym, rel in vdprelations.items():
#     if 'labelOf' in rel_sym:
#         rel_interpretation[rel] = {(obj, lab) : rel_sym in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}
#     else:
#         rel_interpretation[rel] = {(obj1, obj2) : rel_sym in relations[(obj1_sym, obj2_sym)]  for obj1_sym, obj1 in vdpobjects.items() for obj2_sym, obj2 in vdpobjects.items() if obj1_sym != obj2_sym}

        
rel_interpretation[vdprelations['labelOf']] = {(obj, lab) : 'labelOf' in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}


rel_interpretation

{<vdp.vocabulary.FOFunction at 0x10889ad10>: {(<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a026d0>): True,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02610>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02810>): True,
  (<vdp.fomodel.FOElement at 0x111a02710>,
   <vdp.fomodel.FOElement at 0x111a02910>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a026d0>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02810>): False,
  (<vdp.fomodel.FOElement at 0x111a02790>,
   <vdp.fomodel.FOElement at 0x111a02910>): True}}

In [15]:
vdpinterpretation = {**labelconst_interpretation, **rel_interpretation}


In [16]:
model = FOModel(vdpvocabulary, vdpelements, vdpinterpretation)


In [17]:
tst = FOElement('x', vdpobject)
model.interpret(term=[vdprelations['labelOf'], [tst], [vdpconstants['cat']]], interpretation_extension={tst: vdpobjects['cat']})

True

In [19]:
{i : c for i, c in enumerate(vdp.utils._read_txt("darknet/data/coco.names"))}

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorbike',
 4: 'aeroplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'sofa',
 58: 'pottedplant',
 59: 'bed',
 60: 'diningtable',
 61: 'toilet',
 62: 'tvmonitor',
 63: 'laptop',
 64: 

In [16]:
def dist(c1, c2):
    """Measure euclidean distance between centroids"""
    return ((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)**0.5
def frame(rc):
    """Return the left, up, right, down `bounds` given the center, width, and height of img"""
    left  = rc['center_x'] - 0.5 * rc['width']
    up    = rc['center_y'] + 0.5*  rc['height']
    right = rc['center_x'] + 0.5*  rc['width']
    down  = rc['center_y'] - 0.5 * rc['height']
    return (left, up, right, down) # Following YOLO syntax

def relate(c1, c2):
    """Relate position of c1 w.r.t c2. `c1 is to the ___ of c2`"""
    x_rel = "right" if c1[0] >= c2[0] else "left"
    y_rel = "above" if c1[1] >= c2[1] else "below"
    return (x_rel, y_rel)
    
def is_within(bb1, b22):
    """Whether bb1 is within bb2."""
    return (bb1[0] >= bb2[0]) and (bb1[2] <= bb2[2]) and (bb1[1] >= bb2[1]) and (bb1[3] <= bb2[3])


In [4]:
import vdp

preds = vdp.utils._read_json("./data/yolo_processed/run_on_2021_01_17_07_55/predictions.json")

for img_pred in preds:
    rel_labels = list()
    var_const_map = dict()
    
    object_id = {obj : str(idx) for idx, obj in enumerate(img_data.keys())}
    for obj1, obj_data1 in img_data.items():
        for obj2, obj_data2 in img_data.items():
            if obj1 != obj2:
                c1 = centroid(obj_data1['bb'])
                c2 = centroid(obj_data2['bb'])
                score = obj_data1['score'] * obj_data2['score'] * 1e-4
                relations = relate(c1, c2)
                if is_within(obj_data1['bb'], obj_data2['bb']):
                    relations.append('within')
                for rel in relations:
                    rel_labels.append([object_id[obj1], obj1, object_id[obj2], obj2, rel, str(score)])
        var_const_map[object_id[obj1] + "_" + obj1] = obj1

    fo_model = _construct_normalized_model(rel_labels, var_const_map)
    img_path =  yolo_input_dir + f"/{img_name}.jpg"
    fo_models.append((img_path, fo_model))
    
return fo_models